In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from google.colab import files

print("✅ Setup Complete. I have inmported all the necessary libraries important for our model building")

✅ Setup Complete. I have inmported all the necessary libraries important for our model building


### Now for Cell 2 , we are genearting synthetic data : Purpose: To create a realistic, multi-featured dataset from scratch and save it as a CSV file.

In [ ]:
import pandas as pd
import numpy as np
import time

# --- 1. Configuration ---
num_points = 20000  # Generate 20,000 data points
num_cycles = 5      # Simulate 5 full charge/discharge cycles
points_per_cycle = num_points // num_cycles

print(f"--- Generating {num_points} data points across {num_cycles} cycles ---")

# --- 2. Initialize Empty Lists for Data ---
all_battery = []
all_charging_status = []

# --- 3. Simulate Multiple Charge/Discharge Cycles ---
for i in range(num_cycles):
    # a) Simulate a discharge phase (100% -> 15%)
    discharge_points = int(points_per_cycle * 0.75) # 75% of the time is spent discharging
    discharge_battery = np.linspace(100, 15, discharge_points)
    all_battery.extend(discharge_battery)
    all_charging_status.extend(np.zeros(discharge_points)) # 0 = discharging

    # b) Simulate a charge phase (15% -> 100%)
    charge_points = points_per_cycle - discharge_points # 25% of the time is spent charging
    charge_battery = np.linspace(15, 100, charge_points)
    all_battery.extend(charge_battery)
    all_charging_status.extend(np.ones(charge_points)) # 1 = charging

# Convert lists to numpy arrays
battery = np.array(all_battery)
charging_status = np.array(all_charging_status)

# Add realistic noise to the battery data
battery += np.random.normal(0, 0.5, len(battery))
battery = np.clip(battery, 0, 100)

print("--- Step 1/4: Battery cycles simulated ---")


# --- 4. Simulate Realistic Brightness ---
# Create a day/night cycle using a sine wave
day_night_cycle = (np.sin(np.linspace(0, num_cycles * 2 * np.pi, len(battery))) + 1) / 2  # Scaled to 0-1
brightness = 30 + (day_night_cycle * 200) # Varies between 30 (night) and 230 (day)

# Add random user activity spikes
activity_spikes = np.random.randint(0, 30, len(battery))
brightness += activity_spikes

# When charging for a long time (e.g., overnight), assume screen is mostly off (low brightness)
for i in range(1, len(brightness)):
    if charging_status[i] == 1 and charging_status[i-1] == 1: # If it's still charging
        brightness[i] = min(brightness[i], 20) # Screen is likely off

brightness = np.clip(brightness, 10, 255)

print("--- Step 2/4: Brightness patterns simulated ---")

# --- 5. Simulate Realistic Network Status ---
network = np.ones(len(battery)) # Default to WiFi (1)
# When discharging and battery is lower, higher chance of being on Mobile Data (2)
mobile_mask = (charging_status == 0) & (battery < 70) & (np.random.rand(len(battery)) > 0.5)
network[mobile_mask] = 2

print("--- Step 3/4: Network switching simulated ---")


# --- 6. Create Timestamp and Assemble Final DataFrame ---
current_timestamp_ms = int(time.time() * 1000)
timestamps = [current_timestamp_ms - (i * 60 * 1000) for i in range(len(battery))] # One data point per minute
timestamps.reverse()

df = pd.DataFrame({
    'timestamp': timestamps,
    'battery': battery,
    'brightness': brightness,
    'network': network,
    'charging_status': charging_status
})

# Save the final dataset
df.to_csv('large_battery_data.csv', index=False, header=False)

print("--- Step 4/4: Final dataset 'large_battery_data.csv' created successfully! ---")
df.head()

--- Generating 20000 data points across 5 cycles ---
--- Step 1/4: Battery cycles simulated ---
--- Step 2/4: Brightness patterns simulated ---
--- Step 3/4: Network switching simulated ---
--- Step 4/4: Final dataset 'large_battery_data.csv' created successfully! ---


,timestamp,battery,brightness,network,charging_status
0,1755130379591,100.000000,158.000000,1.0,0.0
1,1755130439591,99.859296,158.157087,1.0,0.0
2,1755130499591,100.000000,149.314174,1.0,0.0
3,1755130559591,99.903564,137.471261,1.0,0.0
4,1755130619591,99.431104,149.628346,1.0,0.0


# Loading and Normalizing the data we have prepared

In [ ]:
# Load the data from the CSV file
df = pd.read_csv('large_battery_data.csv', header=None, names=['timestamp', 'battery', 'brightness', 'network' , 'charging_status'])

# Normalize the features to be between 0 and 1
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df[['battery', 'brightness', 'network', 'charging_status']])

print("✅ Data loaded and normalized successfully!")

✅ Data loaded and normalized successfully!


# Now preparing the Data for the training of AI mdoel

In [ ]:
# This function converts our data into sequences (e.g., use data from steps 1-10 to predict step 11)
def create_dataset(dataset, look_back=10):
    dataX, dataY = [], []
    for i in range(len(dataset) - look_back - 1):
        a = dataset[i:(i + look_back), :]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

look_back = 10
X, y = create_dataset(df_scaled)

print(f"✅ Data prepared for AI training. Input shape: {X.shape}, Output shape: {y.shape}")

✅ Data prepared for AI training. Input shape: (19989, 10, 4), Output shape: (19989,)


## Training and Building the AI model

In [ ]:
# Cell 5 (NEW): Build and Train a Simpler, More Robust Model

num_features = X.shape[2]
look_back = 10

model = tf.keras.Sequential([
    # This layer flattens the input from a (10, 4) shape into a single line of 40 numbers
    tf.keras.layers.Flatten(input_shape=(look_back, num_features)),

    # These are standard "Dense" layers that are very robust
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),

    # The final output layer is still a single neuron
    tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error')

print("⏳ Training the new, more robust model...")
model.fit(X, y, epochs=10, batch_size=1, verbose=2)
print("✅ Training complete!")

⏳ Training the new, more robust model...
Epoch 1/10


/usr/local/lib/python3.12/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


19989/19989 - 35s - 2ms/step - loss: 4.8570e-04
Epoch 2/10
19989/19989 - 34s - 2ms/step - loss: 1.0658e-04
Epoch 3/10
19989/19989 - 38s - 2ms/step - loss: 8.6606e-05
Epoch 4/10
19989/19989 - 31s - 2ms/step - loss: 7.9944e-05
Epoch 5/10
19989/19989 - 31s - 2ms/step - loss: 7.8816e-05
Epoch 6/10
19989/19989 - 39s - 2ms/step - loss: 7.4759e-05
Epoch 7/10
19989/19989 - 44s - 2ms/step - loss: 7.4564e-05
Epoch 8/10
19989/19989 - 29s - 1ms/step - loss: 7.3970e-05
Epoch 9/10
19989/19989 - 28s - 1ms/step - loss: 7.3040e-05
Epoch 10/10
19989/19989 - 30s - 1ms/step - loss: 7.2146e-05
✅ Training complete!


Converting and Downloading the mobile friendly model


In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS,
  tf.lite.OpsSet.SELECT_TF_OPS
]
converter._experimental_lower_tensor_list_ops = False

tflite_model = converter.convert()

with open('advanced_battery_model.tflite', 'wb') as f:
  f.write(tflite_model)

print("\n⬇️ Downloading your 'advanced_battery_model.tflite' file...")
files.download('advanced_battery_model.tflite')

Saved artifact at '/tmp/tmpn4vmkjzl'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 10, 4), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  139471186994064: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139471186995024: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139471186994448: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139471186994832: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139471186995216: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139471186993104: TensorSpec(shape=(), dtype=tf.resource, name=None)

⬇️ Downloading your 'advanced_battery_model.tflite' file...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>